In [67]:
import pandas as pd
import subprocess
import datetime as dt

In [68]:
image_file = pd.read_csv('new_image_bbox.csv')

In [69]:
## Retrieve tar files from the cyverse data store for specified season and day ex: ('season_10_yr_2020', '2020-03-03')
class Thermal:
    def get_tar(season, date):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/level_1/{season}/EnvironmentLogger/{date}_clean.tar.gz'
        subprocess.call(command, shell = True)
        command = f'tar -xvf {date}_clean.tar.gz'
        subprocess.call(command, shell = True)

## Retrieve csv data and organize/clean up
    def retrieve_data(season, date):
        tar_data = Thermal.get_tar(season, date)
        data = pd.read_csv(f'./{date}_clean.csv')
        data['Time'] = pd.to_datetime(data['Time'])
        #data['date'] = data.Time.dt.date
        #data['time'] = data.Time.dt.time
        data_clean = data[['Time', 'Sun Direction', 'Temperature', 
                                      'Photosynthetically active radiation', 'Wind velocity']]
        #index_data = data_clean.set_index('time')
        return data_clean

In [70]:
## Searches an already downloaded file of the year's AZmet data
## Has the capability of retrieving weather data for the specific date and hour

## Labels the columns with their appropriate header
class AZmet:
    def __init__ (self, filename):
        file = pd.read_csv(filename, names = ["Year", "Day", "Hour", 
                                            "Air Temperature", "Relative Humidity", 
                                            "VPD", "Solar Radiation", "Precipitation", 
                                            "4 inch Soil T", "12 inch Soil T", 
                                            "Avg Wind Speed", "Wind Vector Magnitude", 
                                            "Wind Vector Direction", "Wind Direction STDEV", 
                                            "Max Wind Speed", "Reference Evapotranspiration", 
                                            "Actual Vapor Pressure", "Dewpoint"])
        file_df = pd.DataFrame(file)
        file_df['combined'] = file_df["Year"]*1000 + file_df["Day"]
        file_df["date"] = pd.to_datetime(file_df["combined"], format = "%Y%j")
        data = file_df.set_index('date')
        del data['combined']
        self.__data__ = data

## Selects the data for the specific Date
    def specific_time(self, date, hour):
        data = self.__data__
        specific_time = data[data.index == date]
        return specific_time

## Selects the data for the specific hour
    def specific_hour(self, date, hour):
        date_data = AZmet.specific_time(self, date, hour)
        hour = date_data[date_data["Hour"] == hour]
        return hour

## Insert AZmet file for that year here
filename = AZmet('/Users/sebastiancalleja/Desktop/AZmet_Data/2020_azmet_data.txt')

In [71]:
## Environmental logger data for specified date
EL = Thermal.retrieve_data('season_10_yr_2020', '2020-03-03')

In [72]:
## azmet data for specified date
AZmet = pd.read_csv('/Users/sebastiancalleja/Desktop/AZmet_Data/azmet_2020_03-03.csv')

In [73]:
## Converts azmet date to datetime
AZmet['date'] = pd.to_datetime(AZmet['date'])

In [74]:
image_file['Date and Time'] = pd.to_datetime(image_file['Date and Time'])

In [75]:
## Gathers environmental logger data for each plot based on 'nearest' HH:MM time
Env_temp = []
def env_temp(image_file):
    for i, row in image_file.iterrows():
        time = row['Date and Time']
        result_index = EL['Time'].sub(time).abs().idxmin()
        EL_result = EL['Temperature'].iloc[result_index]
        Env_temp.append(EL_result)
    return (Env_temp)

In [76]:
Env_wind = []
def env_wind(image_file):
    for i, row in image_file.iterrows():
        time = row['Date and Time']
        result_index = EL['Time'].sub(time).abs().idxmin()
        EL_result = EL['Wind velocity'].iloc[result_index]
        Env_wind.append(EL_result)
    return (Env_wind)

In [77]:
image_file['Env Temperature'] = env_temp(image_file)

In [78]:
image_file['Env Wind Velocity'] = env_wind(image_file)

In [79]:
## Gathers azmet data for each plot based on 'nearest' HH: time
AZmet_temp = []
def azmet_temp(image_file):
    for i, row in image_file.iterrows():
        time = row['Date and Time']
        result_index = AZmet['date'].sub(time).abs().idxmin()
        AZmet_result = AZmet['Air Temperature'].iloc[result_index]
        AZmet_temp.append(AZmet_result)
    return (AZmet_temp)

In [80]:
AZmet_wind = []
def azmet_wind(image_file):
    for i, row in image_file.iterrows():
        time = row['Date and Time']
        result_index = AZmet['date'].sub(time).abs().idxmin()
        AZmet_result = AZmet['Avg Wind Speed'].iloc[result_index]
        AZmet_wind.append(AZmet_result)
    return (AZmet_wind)

In [29]:
#AZmet_dict = dict()
#def azmet_dict(image_file):
    #for i, row in image_file.iterrows():
        #time = row['Date and Time']
        #result_index = AZmet['date'].sub(time).abs().idxmin()
        #AZmet_temp = AZmet['Air Temperature'].iloc[result_index]
        #AZmet_wind = AZmet['Avg Wind Speed'].iloc[result_index]
        #AZmet_vpd = AZmet['VPD'].iloc[result_index]
        #AZmet_solar = AZmet['Solar Radiation'].iloc[result_index]
        #AZmet_precipitation = AZmet['Precipitation'].iloc[result_index]
        #AZmet_dict[AZmet_temp, AZmet_wind, AZmet_vpd, AZmet_solar, AZmet_precipitation] = "Atm Temp, Avg Wind Speed, VPD, Solar Radiation, Precipitation"
    #return (AZmet_dict)

In [81]:
image_file['azmet temp'] = azmet_temp(image_file)

In [82]:
image_file['azmet wind speed'] = azmet_wind(image_file)

In [83]:
#image_file_i.loc[image_file_i.index.unique()[image_file_i.index.unique().get_loc(time1, method='nearest')]]

In [84]:
#mean = image_file.groupby('Image Name').mean()

In [85]:
#image_file.to_csv('2020-03-03_all_atm_temp.csv')

In [86]:
## Groups file by Image Name and finds the mean
image_file_mean = image_file.groupby('Image Name').mean()

In [87]:
del image_file_mean['Unnamed: 0']

In [88]:
## Sorts values by Image Name
sorted_mean = image_file_mean.sort_values('Image Name')

In [89]:
sorted_mean = sorted_mean.reset_index()

In [90]:
sorted_image_file = image_file.sort_values('Image Name')

In [91]:
del sorted_image_file['Unnamed: 0']

In [92]:
sorted_image_file['Avg EL'] = sorted_mean['Env Temperature']

In [93]:
sorted_image_file['Avg AZmet'] = sorted_mean['azmet temp']

In [94]:
full_image_file = sorted_image_file.sort_values('Date and Time')

In [95]:
#del full_image_file['Unnamed: 0']

In [96]:
full_image_file.to_csv('2020-03-03_all_atm_temp.csv')